# Installation

For installation instructions please refer to `Simply laced diagrams` notebook.

In [1]:
using Pkg

In [2]:
versioninfo()

Julia Version 1.9.1
Commit 147bdf428cd (2023-06-07 08:27 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 16 × AMD Ryzen 7 PRO 4750U with Radeon Graphics
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, znver2)
  Threads: 9 on 16 virtual cores
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_IMAGE_THREADS = 1


In [3]:
Pkg.activate(joinpath(@__DIR__, ".."))

  Activating project at `~/Mathematics/Research/Property (T)/Chevalley/2306.12358`


In [4]:
Pkg.status()

Status `~/Mathematics/Research/Property (T)/Chevalley/2306.12358/Project.toml`
  [1e616198] COSMO v0.8.7
  [5d8bd718] Groups v0.7.7
  [7073ff75] IJulia v1.24.2
  [4076af6c] JuMP v1.12.0
  [03b72c93] PropertyT v0.5.0 `https://github.com/kalmarek/PropertyT.jl#master`
  [c946c3f1] SCS v1.2.1
⌅ [856f044c] MKL_jll v2022.2.0+0 ⚲
  [ade2ca70] Dates
  [37e2e46d] LinearAlgebra
Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`


In [5]:
using Groups
import Groups.MatrixGroups

In [6]:
using PropertyT
using PropertyT.PermutationGroups
using PropertyT.IntervalArithmetic

[ Info: Precompiling PropertyT [03b72c93-0167-51e2-8a1e-eb4ff1fb940d]


> In this notebook only rudimentary commentary is included. For the extended one please consult `Simply laced diagrams` notebook.

# $G₂$ as matrix group

We will first define $G₂$ by explicit matrix generators.

The following [GAP](https://www.gap-system.org/) code has been used to obtain matrices of adjoint operators with respect to a basis of the $\mathfrak{g}_2$ algebra.

```GAP
gap> g2 := SimpleLieAlgebra("G", 2, Rationals);
<Lie algebra of dimension 14 over Rationals>
gap> g2rs := RootSystem(g2);
<root system of rank 2>
gap> positive := PositiveRootVectors(g2rs);
[ v.1, v.2, v.3, v.4, v.5, v.6 ]
gap> negative := NegativeRootVectors(g2rs);
[ v.7, v.8, v.9, v.10, v.11, v.12 ]
gap> all_gens := ShallowCopy(positive);; Append(all_gens, negative);; all_gens;
[ v.1, v.2, v.3, v.4, v.5, v.6, v.7, v.8, v.9, v.10, v.11, v.12 ]
gap> adj_mats := List(all_gens, x->AdjointMatrix(Basis(g2), x));;
```

With this data we can define the group $G_2$ as the matrix group generated by exponentials of `adj_mats`:

In [7]:
include(joinpath(@__DIR__, "..", "src", "G₂_gens.jl"));

In [8]:
mats, _ = G₂_matrices_roots();
mats[1]

14×14 Matrix{Int64}:
 1   0   0   0  0  0  -1  0  0  0  0  0  -2  1
 0   1   0   0  0  0   0  0  0  0  0  0   0  0
 0  -1   1   0  0  0   0  0  0  0  0  0   0  0
 0   1  -2   1  0  0   0  0  0  0  0  0   0  0
 0  -1   3  -3  1  0   0  0  0  0  0  0   0  0
 0   0   0   0  0  1   0  0  0  0  0  0   0  0
 0   0   0   0  0  0   1  0  0  0  0  0   0  0
 0   0   0   0  0  0   0  1  3  3  1  0   0  0
 0   0   0   0  0  0   0  0  1  2  1  0   0  0
 0   0   0   0  0  0   0  0  0  1  1  0   0  0
 0   0   0   0  0  0   0  0  0  0  1  0   0  0
 0   0   0   0  0  0   0  0  0  0  0  1   0  0
 0   0   0   0  0  0   1  0  0  0  0  0   1  0
 0   0   0   0  0  0   0  0  0  0  0  0   0  1

In [9]:
d = size(first(mats), 1)

14

In [10]:
G = MatrixGroups.MatrixGroup{d}(mats)

subgroup of 14×14 invertible matrices with 12 generators

## Weyl group of $G_2$

Finally we define a finite group of automorphisms of $G_2$ which act by permutations on the symmetric generating set. While the classical group is the group of reflections of the root system, our group, also generated by two reflections, is a $\mathbb{Z}/2\mathbb{Z}^n$-extension of the classical Weyl group.

In [11]:
S = let S = Groups.gens(G)
    union!(S, inv.(S)) # symmetric generating set
end

24-element Vector{FPGroupElement{Groups.MatrixGroups.MatrixGroup{14, Int64, DataType, Groups.MatrixGroups.MatrixElt{14, Int64, 196}}, …}}:
 m₁
 m₂
 m₃
 m₄
 m₅
 m₆
 m₇
 m₈
 m₉
 m₁₀
 m₁₁
 m₁₂
 m₁⁻¹
 m₂⁻¹
 m₃⁻¹
 m₄⁻¹
 m₅⁻¹
 m₆⁻¹
 m₇⁻¹
 m₈⁻¹
 m₉⁻¹
 m₁₀⁻¹
 m₁₁⁻¹
 m₁₂⁻¹

In [12]:
σ = let S = S, a = S[1], b = S[7]
    w = a * inv(b) * a
    # w is an element of G₂ which acts on S by conjugation:
    images = [findfirst(==(w^-1 * s * w), S) for s in S]
    PermutationGroups.Perm(images)
end

(1,19)(2,5,14,17)(3,16,15,4)(7,13)(8,11,20,23)(9,22,21,10)

In [13]:
τ = let S = S, a = S[2], b = S[8]
    w = a * inv(b) * a
    # w is an element of G₂ which acts on S by conjugation:
    images = [findfirst(==(w^-1 * s * w), S) for s in S]
    PermutationGroups.Perm(images)
end

(1,15,13,3)(2,20)(5,6,17,18)(7,21,19,9)(8,14)(11,12,23,24)

In [ ]:
Weyl = PermGroup(σ, τ)

In [ ]:
Groups.order(Weyl)

(as compared to the classical Weyl group of order $12$ isomorphic to $D_6$).

# Sum of squares proof of property (T) for $\operatorname{G}_{2}(\mathbb{Z})$

We wish to prove
> **Theorem 3.17** Let $G$ be the universal Chevalley group over $\mathbb{Z}$ of type $\texttt{G}_{\texttt{2}}$ and let $S$ be the set of its Steinberg generators. The pair $(G, S)$ has property (T) with a witness of type $(\lambda, R) = (0.96768, 2)$.

We will show this by exhibiting $\xi_i\in \mathbb{R}G$, supported inside $\operatorname{Ball}(S, 2)$ such that

$$
\Delta^2 - \lambda \Delta - \sum_i \xi_i^* \xi_i = r,
$$

with $\|r\|_1$ much smaller (a few orders of magnitue) than $\lambda$.

In [ ]:
HALFRADIUS = 2
RG, S, sizes = @time PropertyT.group_algebra(G, halfradius = HALFRADIUS);

In [ ]:
Δ = RG(length(S)) - sum(RG(s) for s in S)

## Optimization problem

### Symmetry reduction

In [ ]:
import PropertyT.StarAlgebras
import PropertyT.SymbolicWedderburn
using PropertyT.PermutationGroups

In [ ]:
wd = let Σ = Weyl, RG = RG
    act = PropertyT.AlphabetPermutation{eltype(Σ),Int64}(
        Dict(g => PermutationGroups.perm(g) for g in Σ),
    )

    @time SymbolicWedderburn.WedderburnDecomposition(
        Float64,
        Σ,
        act,
        StarAlgebras.basis(RG),
        StarAlgebras.Basis{UInt16}(@view StarAlgebras.basis(RG)[1:sizes[HALFRADIUS]]),
        semisimple = false,
    )
end
@info wd

In [ ]:
@time model, varP = PropertyT.sos_problem_primal(Δ^2, Δ, wd; augmented = true);
model

## Numerical approximation of the solution

In [ ]:
include(joinpath(dirname(pathof(PropertyT)), "..", "test", "optimizers.jl"));
with_optimizer = cosmo_optimizer(;
    eps = 1e-9,
    max_iters = 20_000,
    accel = 50,
    alpha = 1.95,
);

In [ ]:
warmstarting = nothing

In [ ]:
status, warmstarting = PropertyT.solve(
    model, 
    with_optimizer, 
    warmstarting,
);
@info "Optimization has finished with" status

## Reconstructing the solution and certification 

In [ ]:
@info "reconstructing the solution"
Q = @time let wd = wd, Ps = [JuMP.value.(P) for P in varP]
    Qs = real.(sqrt.(Ps))
    PropertyT.reconstruct(Qs, wd)
end

In [ ]:
@info "certifying the solution"
@time certified, λ = PropertyT.certify_solution(
    Δ^2,
    Δ,
    JuMP.objective_value(model),
    Q;
    halfradius = HALFRADIUS,
    augmented = true,
)
if certified && λ > 0
    Κ(λ, S) = sqrt(2λ/length(S))
    @info "Certified result: G₂ has property (T):" inf(λ) inf(Κ(λ, S))
else
    @info "Could NOT certify property (T) for G₂" certified λ
end

# $\texttt{G}_\texttt{2}$-graded $\operatorname{Adj}$

We wish to prove

> **Theorem 3.18** Let $G$ be the universal Chevalley group over $\mathbb{Z}$ of type $\texttt{G}_\texttt{2}$ and let $S$ be the set of its Steinberg generators. Let $V$ denote the ambient vector space of the root system. Then 
>
> $$\operatorname{Adj}_V −\lambda \Delta_V ⩾_R 0$$
>
>for $(\lambda, R) \in (1.56799, 3)$.


**BIG FAT WARNING**: Proving this theorem for $R = 3$ requires **lots of memory** (`>32GB`) and **even more patience**. This is due to the following facts.
* That there are more than $22\,000\,000$ elements in the ball of radius $6$ while the Weyl groups is fairly small, with just `48` elements. This results in Wedderburn Decomposition into `459849` orbits and `10` simple summands of sizes `[334, 332, 182, 167, 165, 153, 459, 447, 449, 439]` (fairly large PSD constraints).
 * Simply generating the data to formulate the optimization problem takes more than `2h` on a workstation computer.
 * Running `scs` solver on the problem for `100_000` iterations takes more than `24h`.

If you have the necessary technical requirements and enough grit you may change `HALFRADIUS` to `3` below. `HALFRADIUS = 2` will not allow  to obtain a positive result!

In [ ]:
HALFRADIUS = 2
RG, S, sizes = @time PropertyT.group_algebra(G, halfradius = HALFRADIUS);

In [ ]:
Δ = RG(length(S)) - sum(RG(s) for s in S)

## Defining $\texttt{G}_\texttt{2}$-grading

Through GAP, we obtain the set of roots of $G_2$ corresponding to `all_gens` (as above):
```GAP
gap> roots := ShallowCopy(PositiveRoots(g2rs));; Append(roots, NegativeRoots(g2rs));; roots;
[ [ 2, -1 ], [ -3, 2 ], [ -1, 1 ], [ 1, 0 ], [ 3, -1 ], [ 0, 1 ], [ -2, 1 ], [ 3, -2 ], [ 1, -1 ], [ -1, 0 ], [ -3, 1 ], [ 0, -1 ] ]
```

These roots are the ones from the Cartan matrix. To obtain the standard (more hexagonal) picture map them by `T` defined as follows:
```julia
cartan = [ 2 -3 ;
          -1  2 ]
rot(α) = [cos(α) -sin(α); sin(α) cos(α)]

c₁ = [√2, 0]
c₂ = rot(5π / 6) * [√2, 0] * √3 # (= 1/2[√6, 1])

T = hcat(c₁, c₂) * inv(cartan)
```
By plotting one against the others (or by blind calculation) one can see the following assignment:

```julia
G₂roots_gap = [
    [2, -1], # α = e₁ - e₂
    [-3, 2], # A = -α + β = -e₁ + 2e₂ - e₃
    [-1, 1], # β = e₂ - e₃
    [1, 0], # α + β = e₁ - e₃
    [3, -1], # B = 2α + β = 2e₁ - e₂ - e₃
    [0, 1], # A + B = α + 2β = e₁ + e₂ - 2e₃
    [-2, 1], # -α
    [3, -2], # -A
    [1, -1], # -β
    [-1, 0], # -α - β
    [-3, 1], # -B
    [0, -1], # -A - B
]
```

One can see that $\langle \alpha, \beta \rangle_\mathbb{Z} = \texttt{A}_\texttt{2}$ and 
$\langle A, B \rangle_\mathbb{Z} = \frac{\sqrt{3}}{\sqrt{2}}\texttt{A}_\texttt{2}$.

The roots corresponding to our generators are therefore of the following form.

In [ ]:
using PropertyT.Roots
e₁ = PropertyT.Roots.𝕖(3, 1)
e₂ = PropertyT.Roots.𝕖(3, 2)
e₃ = PropertyT.Roots.𝕖(3, 3)

α = e₁ - e₂
β = e₂ - e₃
A = -α + β
B = α + (α + β)

roots = [α, A, β, α + β, B, A + B, -α, -A, -β, -α - β, -B, -A - B]

In [ ]:
G₂grading = let A = alphabet(G), grading = Dict{eltype(A), eltype(roots)}()  
    for (root, g) in zip(roots, gens(G))
        letter = first(word(g))
        # assigning root to both g and g⁻¹:
        grading[A[letter]] = root
        grading[A[inv(letter, A)]] = root
    end
    grading 
end

In [ ]:
function PropertyT.grading(g::MatrixGroups.MatrixElt, grading = G₂grading)
    return grading[g]
end

In [ ]:
g = gens(G,1)

In [ ]:
PropertyT.grading(g)

In [ ]:
PropertyT.grading(inv(g))

In [ ]:
g = gens(G, 2)

In [ ]:
PropertyT.grading(g)

In [ ]:
Δs = PropertyT.laplacians(
    RG,
    S,
    x -> (gx = PropertyT.grading(x); Set([gx, -gx])),
);

Here `Δs` is just a map from lines in the root system $\Omega = \texttt{G}_{\texttt{2}}$ to the corresponding Laplacians, e.g. below we can see that to the line through `α = [1, -1, 0]` and `-α` (and the origin) we assign
$$ \Delta_{Lα} = 4 - m_{1} - m_{7} - m_{1}^{-1} - m_{7}^{-1}.$$ 

In [ ]:
using PropertyT.Roots
let α = Root([1,-1, 0])
    Lα = Set([α, -α])
    Δs[Lα]
end

In [ ]:
using PropertyT.Roots
let α = Root([-1,2, -1])
    Lα = Set([α, -α])
    Δs[Lα]
end

Following the definition of $\operatorname{Adj}$ we define
$$ \operatorname{Adj}_{\texttt{G}_\texttt{2}} = 
\prod_{
    \langle L\alpha, L\beta \rangle \cap \Omega \cong \texttt{G}_{\texttt{2}}
} \Delta_{L\alpha} \Delta_{L\beta} $$

In [ ]:
AdjG₂ = PropertyT.Adj(Δs, :G₂)

It is not hard to see that for $\Omega = \texttt{G}_{\texttt{2}}$ 
 * we are simply looking at products of all $\Delta_{L\alpha}$ and $\Delta_{L\beta}$ where $L\alpha \neq L\beta$, and
 * that the new definition is an analouge to the definition of $\operatorname{Adj}$ from [On property (T) for $\operatorname{Aut}(F_n)$ and $\operatorname{SL}_n(\mathbb{Z})$](https://arxiv.org/abs/1812.03456).

In [ ]:
AdjG₂ == Δ^2 - sum(Δs[Lα]^2 for Lα in keys(Δs))

## Optimization problem
### Symmetry reduction


In [ ]:
import PropertyT.StarAlgebras
import PropertyT.SymbolicWedderburn
using PropertyT.PermutationGroups

In [ ]:
wd = let Σ = Weyl, RG = RG
    act = PropertyT.AlphabetPermutation{eltype(Σ),Int64}(
        Dict(g => PermutationGroups.perm(g) for g in Σ),
    )

    @time SymbolicWedderburn.WedderburnDecomposition(
        Float64,
        Σ,
        act,
        StarAlgebras.basis(RG),
        StarAlgebras.Basis{UInt16}(@view StarAlgebras.basis(RG)[1:sizes[HALFRADIUS]]),
        semisimple = false,
    )
end
@info wd

In [ ]:
@time model, varP = PropertyT.sos_problem_primal(AdjG₂, Δ, wd; augmented = true);
model

## Solving the problem numerically
We will use `scs` [Splitting Conic Solver](https://github.com/cvxgrp/scs) so solve this problem.

In [ ]:
using MKL_jll
include(joinpath(@__DIR__, "..", "src", "optimizers.jl"));
with_optimizer = scs_optimizer(;
    linear_solver = SCS.MKLDirectSolver,
    eps = 1e-9,
    max_iters = 100_000,
    accel = 50,
    alpha = 1.95,
);

In [ ]:
warm = nothing

> **Note** If you survived until now with `HALFRADIUS = 3`...
> * To obtain just **any positive lower bound** it is advisable to (artificially) bound the objective from above, e.g. by adding
    ```julia
    JuMP.@constraint(model, upper_bound, model[:λ] ≤ 1.0)
    ```
    before solving the model (to bring the solve time to below 1h).
> * If you do not bound the objective you will need to re-run the cell below several (a dozen? times to obtain `status = OPTIMAL::TerminationStatusCode = 1`. To succesfully certify **a lower bound** that might not be necessary. However this will be necessary to obtain **the bound advertised** in the paper.


In [ ]:
if HALFRADIUS == 3
    JuMP.@constraint(model, upper_bound, model[:λ] ≤ 1.0)
end

In [ ]:
status, warm = PropertyT.solve(
        model,
        with_optimizer,
        warm,
    );
# note: since we're using scs there will be no printout until the optimization has finished 
# please bear with us...
@info "Optimization has finished with" status

### Reconstructing and certifying the solution

In [ ]:
@info "reconstructing the solution"
Q = @time let wd = wd, Ps = [JuMP.value.(P) for P in varP]
    Qs = real.(sqrt.(Ps))
    PropertyT.reconstruct(Qs, wd)
end

In [ ]:
@info "certifying the solution"
certified, λ = PropertyT.certify_solution(
    AdjG₂,
    Δ,
    JuMP.objective_value(model),
    Q;
    halfradius = HALFRADIUS,
    augmented = true,
)

if certified && λ > 0
    @info "Certified result: Adj_C₂ is positive" PropertyT.IntervalArithmetic.inf(λ)
else
    @info "Could NOT certify the positivity of Adj_C₂" certified λ
end

If you solved the problem with `HALFRADIUS = 2`, then you might notice that the optimal $\lambda$ that we obtained is `-0.881...` i.e. **negative**. This means that not only $\operatorname{Adj}_{\texttt{G}_{\texttt{2}}}$ is not positive (is not a sum of squares), but one has to **add** almost a whole $\Delta$ to obtain a positive element. In other words

$$
\operatorname{Adj}_{\texttt{G}_{\texttt{2}}} + 0.881...\Delta = \sum_i \xi_i^* \xi_i, \quad \operatorname{supp}{\xi_i} \subseteq \operatorname{Ball}(S, 2).
$$

Passing to `HALFRADIUS = 3` allows us to obtain a positive result i.e.
$$
\operatorname{Adj}_{\texttt{G}_{\texttt{2}}} - 1.568...\Delta = \sum_i \xi_i^* \xi_i, \quad \operatorname{supp}{\xi_i} \subseteq \operatorname{Ball}(S, 3).
$$

In [ ]:
using Dates
Dates.now()

In [ ]:
versioninfo()